Mohammed elidrissi laoukili
* subjet  : video analysis

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# %%
"""
TimeSformer for Video Emotion Recognition using Hugging Face Transformers
This approach uses pretrained TimeSformer from Facebook Research via HuggingFace
"""
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cv2
from datasets import load_dataset
import tempfile
from transformers import TimesformerModel, TimesformerConfig

# %%
# ========================
# 1. DATASET CLASS
# ========================

class SyntheticEmotionsDataset(Dataset):
    """Dataset wrapper for aadityaubhat/synthetic-emotions"""
    def __init__(self, split='train', num_frames=8, frame_size=224, transform=None):
        self.num_frames = num_frames
        self.frame_size = frame_size
        self.transform = transform
        
        print(f"Loading dataset (split: {split})...")
        dataset = load_dataset("aadityaubhat/synthetic-emotions", split=split)
        self.dataset = dataset
        
        self.samples = []
        self.emotion_counts = {}
        
        labels = dataset['label'] if 'label' in dataset.column_names else dataset['emotion']
        
        for idx, emotion in enumerate(labels):
            self.samples.append((idx, emotion))
            self.emotion_counts[emotion] = self.emotion_counts.get(emotion, 0) + 1
        
        self.emotions = sorted(list(set([s[1] for s in self.samples])))
        self.emotion_to_idx = {emotion: idx for idx, emotion in enumerate(self.emotions)}
        
        print(f"Loaded {len(self.samples)} videos")
        print(f"Emotions: {self.emotions}")
        print(f"Distribution: {self.emotion_counts}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample_idx, emotion = self.samples[idx]
        
        try:
            item = self.dataset[sample_idx]
            video_data = item['video']
            frames = self.load_video(video_data)
        except Exception as e:
            frames = torch.zeros(self.num_frames, 3, self.frame_size, self.frame_size)
        
        if self.transform:
            frames = self.transform(frames)
        
        label = self.emotion_to_idx[emotion]
        return frames, label
    
    def load_video(self, video_data):
        """Load and process video"""
        with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp:
            if isinstance(video_data, dict) and 'bytes' in video_data:
                tmp.write(video_data['bytes'])
            elif isinstance(video_data, bytes):
                tmp.write(video_data)
            tmp_path = tmp.name
        
        frames = self.extract_frames(tmp_path)
        
        try:
            os.unlink(tmp_path)
        except:
            pass
        
        return frames
    
    def extract_frames(self, video_path):
        """Extract frames from video"""
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames == 0:
            cap.release()
            # Return shape: [num_frames, 3, H, W]
            return torch.randn(self.num_frames, 3, self.frame_size, self.frame_size)
        
        frame_indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)
        
        frames = []
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (self.frame_size, self.frame_size))
                frame = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
                frames.append(frame)
        
        cap.release()
        
        while len(frames) < self.num_frames:
            if frames:
                frames.append(frames[-1].clone())
            else:
                frames.append(torch.zeros(3, self.frame_size, self.frame_size))
        
        # Stack to [num_frames, 3, H, W]
        frames = torch.stack(frames[:self.num_frames])
        return frames


# %%
# ========================
# 2. DATA AUGMENTATION
# ========================

class VideoTransform:
    """Video augmentation"""
    def __init__(self, mode='train'):
        self.mode = mode
        
    def __call__(self, frames):
        """frames: [T, C, H, W]"""
        if self.mode == 'train':
            # Random horizontal flip
            if torch.rand(1) > 0.5:
                frames = torch.flip(frames, dims=[3])
            
            # Brightness
            brightness = 0.8 + torch.rand(1) * 0.4
            frames = torch.clamp(frames * brightness, 0, 1)
        
        # Normalize (ImageNet stats)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
        frames = (frames - mean) / std
        
        return frames


# %%
# ========================
# 3. TIMESFORMER MODEL (HUGGING FACE)
# ========================

class TimeSformerForEmotionRecognition(nn.Module):
    """
    TimeSformer model using Hugging Face Transformers
    - Uses pretrained weights from Facebook Research
    - Implements divided space-time attention
    """
    def __init__(self, num_classes=10, num_frames=8, pretrained=True):
        super().__init__()
        
        print("\n🏗️  Building TimeSformer model...")
        
        if pretrained:
            print("   Loading pretrained TimeSformer from Facebook Research...")
            # Load pretrained TimeSformer (base model)
            self.timesformer = TimesformerModel.from_pretrained(
                "facebook/timesformer-base-finetuned-k400",
                ignore_mismatched_sizes=True
            )
            print("   ✓ Pretrained weights loaded from Kinetics-400")
        else:
            print("   Initializing TimeSformer from scratch...")
            config = TimesformerConfig(
                image_size=224,
                patch_size=16,
                num_channels=3,
                num_frames=num_frames,
                hidden_size=768,
                num_hidden_layers=12,
                num_attention_heads=12,
                intermediate_size=3072,
                attention_type="divided_space_time"
            )
            self.timesformer = TimesformerModel(config)
            print("   ✓ Model initialized from scratch")
        
        # Get hidden size
        hidden_size = self.timesformer.config.hidden_size
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size // 2, num_classes)
        )
        
        print(f"   Architecture:")
        print(f"   - Attention Type: Divided Space-Time")
        print(f"   - Hidden Size: {hidden_size}")
        print(f"   - Transformer Layers: {self.timesformer.config.num_hidden_layers}")
        print(f"   - Attention Heads: {self.timesformer.config.num_attention_heads}")
        print(f"   - Output Classes: {num_classes}")
    
    def forward(self, pixel_values):
        """
        Args:
            pixel_values: [batch_size, num_frames, num_channels, height, width]
        """
        # HuggingFace TimeSformer expects: [batch_size, num_frames, num_channels, height, width]
        # Our dataloader gives: [batch_size, num_frames, num_channels, height, width]
        # So we DON'T need to permute - it's already correct!
        
        # Get TimeSformer outputs
        outputs = self.timesformer(pixel_values)
        
        # Use the [CLS] token representation
        sequence_output = outputs.last_hidden_state
        cls_token = sequence_output[:, 0]  # [batch_size, hidden_size]
        
        # Classification
        logits = self.classifier(cls_token)
        
        return logits


# %%
# ========================
# 4. TRAINING FUNCTIONS
# ========================

def train_epoch(model, loader, criterion, optimizer, device, grad_clip=1.0):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for videos, labels in pbar:
        videos = videos.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return total_loss / len(loader), correct / total


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for videos, labels in tqdm(loader, desc='Validating'):
            videos = videos.to(device)
            labels = labels.to(device)
            
            outputs = model(videos)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    return total_loss / len(loader), acc, f1, all_preds, all_labels


# %%
# ========================
# 5. MAIN SCRIPT
# ========================

def main():
    print("="*80)
    print("TIMESFORMER VIDEO EMOTION RECOGNITION")
    print("Using Hugging Face Transformers + Facebook Research Pretrained Model")
    print("="*80)
    
    config = {
        'num_frames': 8,
        'frame_size': 224,
        'batch_size': 2,
        'num_epochs': 5,
        'learning_rate': 3e-5,
        'weight_decay': 0.01,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_pretrained': True,  # Use pretrained weights
        'train_split': 0.6,
        'val_split': 0.2,
        'test_split': 0.2
    }
    
    print(f"\nConfiguration:")
    for k, v in config.items():
        print(f"  {k}: {v}")
    print("="*80)
    
    # Set seeds
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Load dataset
    print("\n📁 Loading dataset...")
    train_dataset = SyntheticEmotionsDataset(
        split='train',
        num_frames=config['num_frames'],
        frame_size=config['frame_size'],
        transform=VideoTransform(mode='train')
    )
    
    val_dataset = SyntheticEmotionsDataset(
        split='train',
        num_frames=config['num_frames'],
        frame_size=config['frame_size'],
        transform=VideoTransform(mode='val')
    )
    
    # Split data
    dataset_size = len(train_dataset)
    indices = list(range(dataset_size))
    np.random.shuffle(indices)
    
    train_size = int(config['train_split'] * dataset_size)
    val_size = int(config['val_split'] * dataset_size)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size+val_size]
    test_indices = indices[train_size+val_size:]
    
    print(f"\n📊 Splits: Train={len(train_indices)}, Val={len(val_indices)}, Test={len(test_indices)}")
    
    train_subset = Subset(train_dataset, train_indices)
    val_subset = Subset(val_dataset, val_indices)
    test_subset = Subset(val_dataset, test_indices)
    
    train_loader = DataLoader(train_subset, batch_size=config['batch_size'], 
                             shuffle=True, num_workers=0, drop_last=True)
    val_loader = DataLoader(val_subset, batch_size=config['batch_size'], 
                           shuffle=False, num_workers=0)
    test_loader = DataLoader(test_subset, batch_size=config['batch_size'], 
                            shuffle=False, num_workers=0)
    
    # Create model
    model = TimeSformerForEmotionRecognition(
        num_classes=len(train_dataset.emotions),
        num_frames=config['num_frames'],
        pretrained=config['use_pretrained']
    ).to(config['device'])
    
    total_params = sum(p.numel() for p in model.parameters()) / 1e6
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
    print(f"\n📈 Parameters: {total_params:.2f}M total, {trainable_params:.2f}M trainable")
    
    # Training setup
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], 
                           weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['num_epochs'])
    
    # Training loop
    best_val_acc = 0
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    print("\n" + "="*80)
    print("🚀 STARTING TRAINING")
    print("="*80)
    
    for epoch in range(config['num_epochs']):
        print(f"\n📅 Epoch {epoch+1}/{config['num_epochs']}")
        
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config['device']
        )
        
        val_loss, val_acc, val_f1, _, _ = validate(
            model, val_loader, criterion, config['device']
        )
        
        scheduler.step()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.4f}")
        print(f"Val:   Loss={val_loss:.4f}, Acc={val_acc:.4f}, F1={val_f1:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_timesformer_hf.pth')
            print(f"✅ New best model saved!")
    
    # Test evaluation
    print("\n" + "="*80)
    print("🎯 FINAL TEST EVALUATION")
    print("="*80)
    
    model.load_state_dict(torch.load('best_timesformer_hf.pth'))
    test_loss, test_acc, test_f1, test_preds, test_labels = validate(
        model, test_loader, criterion, config['device']
    )
    
    print(f"\n📊 Test Results:")
    print(f"  Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"  F1-Score: {test_f1:.4f}")
    
    print("\n" + "="*80)
    print("✅ TRAINING COMPLETE!")
    print("="*80)


if __name__ == '__main__':
    main()

2026-01-02 11:26:21.102929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767353181.324237      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767353181.387863      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Train: Loss=2.3334, Acc=0.0667
Val:   Loss=2.5483, Acc=0.0500, F1=0.0048

📅 Epoch 5/5


Validating: 100%|██████████| 10/10 [00:02<00:00,  3.82it/s]


Train: Loss=2.3106, Acc=0.1500
Val:   Loss=2.5442, Acc=0.0500, F1=0.0048

🎯 FINAL TEST EVALUATION


Validating: 100%|██████████| 10/10 [00:02<00:00,  3.82it/s]


📊 Test Results:
  Accuracy: 0.0500 (5.00%)
  F1-Score: 0.0048

✅ TRAINING COMPLETE!
